## Berechnungen

In diesem Notebook ist es möglich den Mittelwert, Varianz und Standardabeichung für Messwerte zu berechnen.   
Zudem wird die Gieränderung in rad bestimmt. Die Ergebnisse können zurück in die Merkmal CSV Datei geschrieben werden.

In [1]:
import pandas
import numpy as np
import numpy.testing as npt
import math
import os

In [37]:
#os.remove("merkmaleRoh.csv")

In [3]:
featuresDf = pandas.read_csv("merkmaleRoh.csv")
featuresDf.describe()

,Zeitstempel,Breitengrad,Laengengrad,Geschwindigkeit,Messwerte,StartBewegungsD,StartBelichtung,Belichtungszeit
count,4.164000e+03,4164.000000,4164.000000,4164.000000,4164.0,4.164000e+03,4.164000e+03,4.164000e+03
mean,1.520233e+12,53.613483,10.137698,0.007556,10.0,2.264945e+14,2.264947e+14,3.279630e+07
std,4.067961e+05,0.000021,0.000039,0.128715,0.0,4.067913e+11,4.067962e+11,4.655649e+05
min,1.520233e+12,53.613420,10.137633,0.000000,10.0,2.257848e+14,2.257849e+14,2.999632e+07
25%,1.520233e+12,53.613472,10.137681,0.000000,10.0,2.261525e+14,2.261526e+14,3.287369e+07
50%,1.520233e+12,53.613490,10.137691,0.000000,10.0,2.265032e+14,2.265034e+14,3.287369e+07
75%,1.520234e+12,53.613500,10.137706,0.000000,10.0,2.268460e+14,2.268462e+14,3.287369e+07
max,1.520234e+12,53.613533,10.137800,3.420000,10.0,2.271917e+14,2.271919e+14,3.287369e+07


In [4]:
featuresDf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4164 entries, 0 to 4163
Data columns (total 15 columns):
Zeitstempel          4164 non-null int64
Breitengrad          4164 non-null float64
Laengengrad          4164 non-null float64
Geschwindigkeit      4164 non-null float64
AccelerometerX       4164 non-null object
AccelerometerY       4164 non-null object
AccelerometerZ       4164 non-null object
Azimuth              4164 non-null object
Nick                 4164 non-null object
Roll                 4164 non-null object
SensorZeitstempel    4164 non-null object
Messwerte            4164 non-null int64
StartBewegungsD      4164 non-null int64
StartBelichtung      4164 non-null int64
Belichtungszeit      4164 non-null int64
dtypes: float64(3), int64(5), object(7)
memory usage: 488.0+ KB


In [5]:
# Zeigt die ersten 5 Reihen Beschleunigungssensordaten der X-Achse 
featuresDf.AccelerometerX.head()

0    -0.1216 -0.21987 -0.34445 -0.4441 -0.50851 -0....
1    -0.03939 -0.07748 -0.07731 -0.10782 -0.10157 -...
2    0.20829 0.19727 0.17314 0.16916 0.12001 0.0347...
3    -0.03367 -0.01161 -0.00928 0.02321 0.03389 -0....
4    -0.00272 0.01315 0.01052 -0.02222 -0.03311 -0....
Name: AccelerometerX, dtype: object

Die folgenden Spalten des Datenframes haben als Type ein Python object.   
AccelerometerX       non-null object   
AccelerometerY       non-null object   
AccelerometerZ       non-null object   
Azimuth              non-null object   
Nick                 non-null object   
Roll                 non-null object   
Implizit haben diese Spalten als Typ Strings. Aber in einem DataFrame werden Strings als    
Python objects erkannt. Um den Datentyp auf float zu casten wird aus den Spalten zunächst ein Liste erstellt.    
Welche dann wiederum als Numpy Array auf den Typ float konvertiert wird.   

In [6]:
accXList = featuresDf.AccelerometerX.str.split(" ").tolist()

In [7]:
# Direkt mit dem Numpy Array ist es nicht möglich die Konvertierung durchzuführen, weil die Arrays von Typ object sind.   
pandas.DataFrame(featuresDf.AccelerometerX.str.split(" ")).values[1]

array([ ['-0.03939', '-0.07748', '-0.07731', '-0.10782', '-0.10157', '-0.11191', '-0.07419', '-0.02871', '0.00768', '0.06744']], dtype=object)

In [8]:
accXList[1] # zeige Liste

['-0.03939',
 '-0.07748',
 '-0.07731',
 '-0.10782',
 '-0.10157',
 '-0.11191',
 '-0.07419',
 '-0.02871',
 '0.00768',
 '0.06744']

In [9]:
np.array(accXList).dtype

dtype('<U8')

In [10]:
np.array(accXList).astype(float).dtype 

dtype('float64')

In [11]:
accXNp = np.array(accXList).astype(float) 
accXDf = pandas.DataFrame(accXNp)
accXDf.describe()

,0,1,2,3,4,5,6,7,8,9
count,4164.000000,4164.000000,4164.000000,4164.000000,4164.000000,4164.000000,4164.000000,4164.000000,4164.000000,4164.000000
mean,-0.000303,-0.000223,-0.000096,-0.000668,-0.000398,-0.001042,-0.000991,-0.000880,-0.000681,-0.000853
std,0.036829,0.036028,0.035620,0.032817,0.033255,0.032321,0.031773,0.030101,0.028765,0.026608
min,-1.802400,-1.824990,-1.781770,-1.532680,-1.455990,-1.379300,-1.195380,-1.002280,-0.905720,-0.939100
25%,-0.010345,-0.009798,-0.010140,-0.009960,-0.009842,-0.010842,-0.010390,-0.010902,-0.010105,-0.010423
50%,0.000340,0.000310,0.000595,-0.000045,0.000450,0.000035,-0.000100,0.000040,-0.000060,-0.000305
75%,0.010220,0.010065,0.010732,0.009282,0.009850,0.009853,0.009923,0.010320,0.009448,0.009822
max,0.699100,0.497990,0.413710,0.330970,0.586560,0.222810,0.228310,0.155470,0.379020,0.212140


In [12]:
# Konvertierung von Object zu Float Werten in einer Zeile 
accYDf = pandas.DataFrame(np.array(featuresDf.AccelerometerY.str.split(" ").tolist()).astype(float))

In [13]:
accYDf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4164 entries, 0 to 4163
Data columns (total 10 columns):
0    4164 non-null float64
1    4164 non-null float64
2    4164 non-null float64
3    4164 non-null float64
4    4164 non-null float64
5    4164 non-null float64
6    4164 non-null float64
7    4164 non-null float64
8    4164 non-null float64
9    4164 non-null float64
dtypes: float64(10)
memory usage: 325.4 KB


In [14]:
accYDf.head(2)

,0,1,2,3,4,5,6,7,8,9
0,-0.07841,-0.09338,-0.07470,-0.07508,-0.06006,-0.07870,-0.06296,-0.03504,-0.02803,0.00822
1,-0.00804,-0.00643,-0.02046,-0.03170,-0.07132,-0.04174,-0.04871,-0.03896,-0.03117,-0.04027


In [15]:
accZDf = pandas.DataFrame(np.array(featuresDf.AccelerometerZ.str.split(" ").tolist()).astype(float))

In [16]:
nickDf = pandas.DataFrame(np.array(featuresDf.Nick.str.split(" ").tolist()).astype(float))

In [17]:
rollDf = pandas.DataFrame(np.array(featuresDf.Roll.str.split(" ").tolist()).astype(float))

In [18]:
azimuthDf = pandas.DataFrame(np.array(featuresDf.Azimuth.str.split(" ").tolist()).astype(float))

In [19]:
accXMean = accXDf.T.mean()
accXMean.head(2)

0   -0.386357
1   -0.054326
dtype: float64

In [20]:
# Neue Spalten werden erstellt und Mittelwerte zugewiesen
featuresDf['MittelX'] = accXMean
featuresDf['MittelY'] = accYDf.T.mean()
featuresDf['MittelZ'] = accZDf.T.mean()
featuresDf['MittelNick'] = nickDf.T.mean()
featuresDf['MittelRoll'] = rollDf.T.mean()

In [21]:
featuresDf.MittelX.head(2)

0   -0.386357
1   -0.054326
Name: MittelX, dtype: float64

In [22]:
featuresDf.columns

Index(['Zeitstempel', 'Breitengrad', 'Laengengrad', 'Geschwindigkeit',
       'AccelerometerX', 'AccelerometerY', 'AccelerometerZ', 'Azimuth', 'Nick',
       'Roll', 'SensorZeitstempel', 'Messwerte', 'StartBewegungsD',
       'StartBelichtung', 'Belichtungszeit', 'MittelX', 'MittelY', 'MittelZ',
       'MittelNick', 'MittelRoll'],
      dtype='object')

Berechnet die Varianz. Dieser Funktion muss als Paramter der Mittelwert (mean) und der DataFrame mit   
den Float Werten übergeben werden, um die Varianz zu berechnen. Als Varianz wird der Durchschnitt der quadrierten  
Differenzen zum Mittelwert bezeichnet.   
Prec.:    
Postc.: Gibt die berechnete Variance als Float zurück oder 0 wenn die Anzahl der Werte <= 0  

In [23]:
def calcVariance(meansDf, dfValues):
    variance = []
    for i, it in dfValues.iterrows():
        sum = 0
        for value in it:
            tempDifference = value-meansDf[i]
            sum += tempDifference * tempDifference
        variance.append("{0:.5f}".format(round(sum / it.count(),5)))
    return np.array(variance).astype(float)

Hier wird die Standardabweichung der Varianz berechnet. Dies ist die Wurzel der Varianz.   
Für die Berechnung wird der Absolutwert von der Varianz genommen.    
Prec.:
Postc.:  Standardabweichung wird zurückgegeben.

In [24]:
def calculateStandardDeviation(varianceDf):
    deviation = []
    for v in varianceDf:
        wasNegative = False
        if(v < 0.0):
            wasNegative = True
        temp = math.sqrt(np.abs(v))
        if(wasNegative):
            temp *= -1
        deviation.append("{0:.5f}".format(round(temp,5)))
    return np.array(deviation).astype(float)

Die Funktion berechnet die Winkeländerung in rad zwischen den ersten gemessenen Gierwinkel und den letzten Gierwinkel   
innerhalb einer als Paramter übergebenen Liste. Der zurückgegebene Radiant    
ist immer positiv und zeigt die relative änderung in rad.      
Prec.:   
Postc.: Radiant berechnet   

In [25]:
def calculateAngelChangeAzimuth(azimuthDf):
    azimuthL = []
    azimuthL.clear
    result = []
    counter = 0
    for i,values in azimuthDf.iterrows():
        resultTemp = 0
        for rad in values:
            azimuthL.append(rad * (180 / math.pi))
            counter = counter + 1
        resultTemp = np.abs(azimuthL[0] - azimuthL[counter-1])
        if(resultTemp > 180):
            resultTemp = 360 - resultTemp
        result.append(resultTemp / (180 * math.pi))
    return np.array(result).astype(float)

In [26]:
featuresDf['AzimuthAenderung'] = calculateAngelChangeAzimuth(azimuthDf)

In [27]:
# Berechne Varianz
featuresDf['VarianzX'] = calcVariance(featuresDf.MittelX,accXDf)
featuresDf['VarianzY'] = calcVariance(featuresDf.MittelY,accYDf)
featuresDf['VarianzZ'] = calcVariance(featuresDf.MittelZ,accZDf)
featuresDf['VarianzNick'] = calcVariance(featuresDf.MittelNick,nickDf)
featuresDf['VarianzRoll'] = calcVariance(featuresDf.MittelRoll,rollDf)

In [28]:
featuresDf.VarianzX.head()

0    0.01530
1    0.00294
2    0.00897
3    0.00058
4    0.00056
Name: VarianzX, dtype: float64

In [29]:
# Berechne Standardabweichung
featuresDf['AbweichungX'] = calculateStandardDeviation(featuresDf.VarianzX)
featuresDf['AbweichungY'] = calculateStandardDeviation(featuresDf.VarianzY)
featuresDf['AbweichungZ'] = calculateStandardDeviation(featuresDf.VarianzZ)
featuresDf['AbweichungNick'] = calculateStandardDeviation(featuresDf.VarianzNick)
featuresDf['AbweichungRoll'] = calculateStandardDeviation(featuresDf.VarianzRoll)

In [30]:
featuresDf.AbweichungX.head()

0    0.12369
1    0.05422
2    0.09471
3    0.02408
4    0.02366
Name: AbweichungX, dtype: float64

In [31]:
accXDf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4164 entries, 0 to 4163
Data columns (total 10 columns):
0    4164 non-null float64
1    4164 non-null float64
2    4164 non-null float64
3    4164 non-null float64
4    4164 non-null float64
5    4164 non-null float64
6    4164 non-null float64
7    4164 non-null float64
8    4164 non-null float64
9    4164 non-null float64
dtypes: float64(10)
memory usage: 325.4 KB


In [32]:
accXDf.head()

,0,1,2,3,4,5,6,7,8,9
0,-0.12160,-0.21987,-0.34445,-0.44410,-0.50851,-0.51407,-0.50319,-0.44852,-0.40479,-0.35447
1,-0.03939,-0.07748,-0.07731,-0.10782,-0.10157,-0.11191,-0.07419,-0.02871,0.00768,0.06744
2,0.20829,0.19727,0.17314,0.16916,0.12001,0.03472,0.01244,-0.02068,-0.01655,-0.04389
3,-0.03367,-0.01161,-0.00928,0.02321,0.03389,-0.00353,-0.00282,0.01307,0.02577,0.05127
4,-0.00272,0.01315,0.01052,-0.02222,-0.03311,-0.04181,-0.04876,-0.05433,-0.01282,0.00506


In [33]:
featuresDf.MittelX.head()

0   -0.386357
1   -0.054326
2    0.083391
3    0.008630
4   -0.018704
Name: MittelX, dtype: float64

In [34]:
# Änderungen in CSV Datei schreiben
featuresDf.to_csv('merkmale.csv')

In [42]:
pandas.read_csv("merkmale.csv").columns

Index(['Unnamed: 0', 'Zeitstempel', 'Breitengrad', 'Laengengrad',
       'Geschwindigkeit', 'AccelerometerX', 'AccelerometerY', 'AccelerometerZ',
       'Azimuth', 'Nick', 'Roll', 'SensorZeitstempel', 'Messwerte',
       'StartBewegungsD', 'StartBelichtung', 'Belichtungszeit', 'MittelX',
       'MittelY', 'MittelZ', 'MittelNick', 'MittelRoll', 'AzimuthAenderung',
       'VarianzX', 'VarianzY', 'VarianzZ', 'VarianzNick', 'VarianzRoll',
       'AbweichungX', 'AbweichungY', 'AbweichungZ', 'AbweichungNick',
       'AbweichungRoll'],
      dtype='object')

In [35]:
# Unittests
import unittest

class CalcTest(unittest.TestCase):
    
    
    # Testet die Funktion calcVariance mit einem DataFrame mit den Werten [2.0, 2.0]
    # Der Mittelwert sollte 2.0 sein
    # Das erwartete Ergebnis ist 0.0
    def testCalcVariance2(self):
        meanL = [2.0]
        meanDf = pandas.DataFrame(meanL)
        valuesA = [2.0,2.0]
        valuesDf = pandas.DataFrame([[2.0]])
        self.assertEqual(calcVariance(meanDf[0],valuesDf), 0.0)
        
    # Testet die Funktion calcVariance mit einem DataFrame mit den folgenden Werten:
    # [1.24, 2.5213, 10.434, 42.45, 5.9]
    # Der Mittelwert ist 12.508
    # Die erwartete Varianz ist 234,247016
    def testCalcVariance5(self):
        meanDf = pandas.DataFrame([12.508])
        valuesDf = pandas.DataFrame([[1.24, 2.52, 10.43, 42.45, 5.9]])
        self.assertEqual(calcVariance(meanDf[0],valuesDf), 234.24702)   
    
    # Testet calcVariance mit 0 Datenwerten   
    def testCalcVariance0(self):
        meanDf = pandas.DataFrame([0.0])
        valuesDf = pandas.DataFrame([[0.0]])
        self.assertEqual(calcVariance(meanDf[0],valuesDf),0.0)  
        
    # Testet calcVariance mit negativen Datenwerten   
    def testCalcVarianceNegative(self):
        meanDf = pandas.DataFrame([-24.0])
        valuesDf = pandas.DataFrame([[-2.0,-24.0,-5.0,7.0]])
        self.assertEqual(calcVariance(meanDf[0],valuesDf),451.5) 
        
    # Testet calcVariance mit Muultidimensionalen DataFrame   
    def testCalcVarianceMultidim(self):
        meanDf = pandas.DataFrame([-24.0,0.0])
        valuesDf = pandas.DataFrame([[-2.0,-24.0,-5.0,7.0],[0.0,0.0,0.0,0.0]])
        # Hier wird Numpy Test verwendet um das Ergebnis (zwei Arrays) zu vergleichen
        npt.assert_array_equal(calcVariance(meanDf[0],valuesDf),np.array([ 451.5,0.0])) 
        
    # Dieser Test testet die Funktion calculateStandardDeviation.
    def testCalculateStandardDeviation(self):
        varianceDf = pandas.DataFrame([451.5])
        self.assertEqual(calculateStandardDeviation(varianceDf[0]),21.24853) 

    # Testet die Funktion calculateStandardDeviation.
    # Dabei ist das Argument negativ.
    def testCalculateStandardDeviationNegative(self):
        varianceDf = pandas.DataFrame([-451.5])
        self.assertEqual(calculateStandardDeviation(varianceDf[0]),-21.24853)
        
    # Testet die Funktion calculateStandardDeviation.
    # Dabei hat das Argument den Wert 0.0.
    def testCalculateStandardDeviationZero(self):
        varianceDf = pandas.DataFrame([0.0])
        self.assertEqual(calculateStandardDeviation(varianceDf[0]),0.0)  
    
unittest.main(argv=[''], verbosity=2, exit=False)

testCalcVariance0 (__main__.CalcTest) ... ok
testCalcVariance2 (__main__.CalcTest) ... ok
testCalcVariance5 (__main__.CalcTest) ... ok
testCalcVarianceMultidim (__main__.CalcTest) ... ok
testCalcVarianceNegative (__main__.CalcTest) ... ok
testCalculateStandardDeviation (__main__.CalcTest) ... ok
testCalculateStandardDeviationNegative (__main__.CalcTest) ... ok
testCalculateStandardDeviationZero (__main__.CalcTest) ... ok

----------------------------------------------------------------------
Ran 8 tests in 0.018s

OK
